In [32]:
import nltk
from nltk.corpus import movie_reviews
import random
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.corpus import wordnet
import string
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
movie_reviews.categories()
lemmatizer=WordNetLemmatizer()

In [3]:
stops=stopwords.words('english')
punc=list(string.punctuation)
stops=stops+punc

In [4]:
len(movie_reviews.fileids())
#1000 -ve
#1000 +ve       

2000

In [5]:
#it returns file id with positive reviews
pos_rev=movie_reviews.fileids('pos')

In [6]:
#we need to pass the file id here to get words
movie_reviews.words(movie_reviews.fileids())

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...]

In [7]:
documents=[]
for category in movie_reviews.categories():
    for fid in movie_reviews.fileids(category):
        documents.append((movie_reviews.words(fid),category))
documents[1]        

(['the', 'happy', 'bastard', "'", 's', 'quick', 'movie', ...], 'neg')

***Shuffling to mix pos/neg so that it becomes easy for train_test_split***

In [8]:
random.shuffle(documents)

In [9]:
def getsimplepos (tag) :
    if tag.startswith( 'J'):
        return wordnet.ADJ
    elif tag.startswith( 'V'):
        return wordnet. VERB
    elif tag.startswith( 'N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet .NOUN

**pos_tag requires an array**

In [10]:
w="better"
pos_tag([w])

[('better', 'RBR')]

**Cleaning the dataset**

In [11]:
def clean_review(words):
    output_words=[]
    for w in words:
        if(w.lower() not in stops):
            pos=pos_tag([w])
            clean_word=lemmatizer.lemmatize(w,pos=getsimplepos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words        
            

In [12]:
documents=[(clean_review(document),category) for document,category in documents]

In [13]:
documents[0]

(['frank',
  'detorri',
  'bill',
  'murray',
  'single',
  'dad',
  'life',
  'beer',
  'junk',
  'food',
  'apparent',
  'understand',
  'sanitation',
  'hygiene',
  'much',
  'dismay',
  'preteen',
  'daughter',
  'shane',
  'elena',
  'franklin',
  'us',
  '10',
  'second',
  'rule',
  'retrieve',
  'hard',
  'boil',
  'egg',
  'chimp',
  'cage',
  'zoo',
  'down',
  'introduces',
  'lethal',
  'bacteria',
  'system',
  'inside',
  'skin',
  'city',
  'frank',
  'turmoil',
  'thanks',
  'vote',
  'pander',
  'mayor',
  'phlegmming',
  'voice',
  'william',
  'shatner',
  'one',
  'frank',
  'pd',
  'white',
  'blood',
  'cell',
  'voice',
  'chris',
  'rock',
  'save',
  'day',
  'peter',
  'bobby',
  'farrelly',
  'osmosis',
  'jones',
  'city',
  'frank',
  'brightly',
  'animate',
  'animation',
  'direct',
  'piet',
  'kroon',
  'tom',
  'sito',
  'cellular',
  'municipality',
  'osmosis',
  'jones',
  'typical',
  'rogue',
  'cop',
  'look',
  'another',
  'chance',
  'inadver

**Now creating dataset with feature as top words**

***Building the feature sets only on training data***

In [14]:
training=documents[:1500]
testing=documents[1500:]

In [16]:
#we will collect all words in an array and find frequency distribution
all_words=[]
for doc in training:
    all_words+=doc[0]

In [17]:
freq=nltk.FreqDist(all_words)
common=freq.most_common(3000)

In [18]:
features=[words for words,count in common]
#features

In [19]:
def get_feature_dict(words):
    current_features={}
    w_set=set(words)
    for w in features:
        current_features[w]=w in w_set
    return current_features    

In [20]:
training_data=[(get_feature_dict(word),category)for word,category in training]
testing_data=[(get_feature_dict(word),category)for word,category in testing]

In [21]:
training_data[0]

({'film': True,
  'movie': False,
  'one': True,
  'make': False,
  'character': False,
  'like': True,
  'get': True,
  'see': False,
  'go': True,
  'time': False,
  'well': True,
  'scene': False,
  'even': False,
  'good': False,
  'story': True,
  'take': False,
  'would': False,
  'much': True,
  'also': False,
  'come': False,
  'bad': False,
  'life': True,
  'way': False,
  'give': False,
  'two': False,
  'look': True,
  'know': False,
  '--': False,
  'first': False,
  'seem': False,
  'end': False,
  'work': False,
  'year': False,
  'thing': False,
  'plot': False,
  'say': False,
  'play': True,
  'little': False,
  'really': False,
  'people': False,
  'show': False,
  'could': False,
  'star': False,
  'love': False,
  'man': False,
  'director': False,
  'never': False,
  'try': False,
  'many': False,
  'best': False,
  'performance': False,
  'new': False,
  'great': False,
  'actor': False,
  'big': False,
  'action': True,
  'u': False,
  'want': False,
  'watch': 

**Using NaiveBayes classifier in nltk**

In [22]:
from nltk import NaiveBayesClassifier

In [23]:
classifier=NaiveBayesClassifier.train(training_data)

In [24]:
nltk.classify.accuracy(classifier,testing_data)

0.792

In [25]:
classifier.show_most_informative_features()

Most Informative Features
            breathtaking = True              pos : neg    =     11.4 : 1.0
             outstanding = True              pos : neg    =     10.2 : 1.0
                  seagal = True              neg : pos    =      9.3 : 1.0
                  random = True              neg : pos    =      8.7 : 1.0
             wonderfully = True              pos : neg    =      8.7 : 1.0
              schumacher = True              neg : pos    =      8.6 : 1.0
               ludicrous = True              neg : pos    =      8.2 : 1.0
                 idiotic = True              neg : pos    =      7.6 : 1.0
             beautifully = True              pos : neg    =      7.4 : 1.0
               pointless = True              neg : pos    =      7.3 : 1.0


**Using Sklearn classifier with the same data**

In [29]:
svc=SVC()
clf=SklearnClassifier(svc)

In [30]:
clf.train(training_data)

<SklearnClassifier(SVC())>

In [31]:
nltk.classify.accuracy(clf,testing_data)

0.85

In [33]:
rfc=RandomForestClassifier()
clf1=SklearnClassifier(rfc)

In [34]:
clf1.train(training_data)

<SklearnClassifier(RandomForestClassifier())>

In [35]:
nltk.classify.accuracy(clf1,testing_data)

0.818